In [1]:
! pip install transformers
! pip install datasets
!pip install torchmetrics
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import f1_score


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 6.9 MB/s 
     |████████████████████████████████| 6.6 MB 48.9 MB/s 
     |████████████████████████████████| 120 kB 75.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 7.3 MB/s 
     |████████████████████████████████| 115 kB 50.2 MB/s 
     |████████████████████████████████| 212 kB 68.4 MB/s 
     |████████████████████████████████| 127 kB 75.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 7.5 MB/s 


In [2]:
from transformers import AutoTokenizer, AutoModel, AdamW
from collections import defaultdict
from transformers import get_linear_schedule_with_warmup
import time

from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics import accuracy_score
from copy import deepcopy

In [3]:
from sklearn.utils import shuffle
import numpy as np

from sklearn.metrics import f1_score
import datasets
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from transformers import BertModel, BertTokenizer
from numpy import *
from torch import nn, optim

In [4]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/新建文件夹/ERAI Dataset_with_translation

Mounted at /content/drive
[Errno 2] No such file or directory: '/content/drive/MyDrive/新建文件夹/ERAI Dataset_with_translation'
/content


In [5]:
pd.set_option('display.max_rows', None)

In [6]:

def generate_input_data(post1,post2,mpp_label,ml_label,max_len,mpp_value1,ml_value1,mpp_value2,ml_value2):
  
  
  first_all_value = pd.concat([mpp_value1,ml_value1],axis=1)
  second_all_value = pd.concat([mpp_value2,ml_value2],axis=1)
  full_text = []
  attention_mask = []
  token_type_ids = []
  first_length = []
  second_length = []
  for i,j in zip(post1,post2):
    if len(i)>max_len:
      i = i[:max_len]
    if len(j)>max_len:
      j = j[:max_len]
    
    i = tokenizer.tokenize(i)
    j = tokenizer.tokenize(j)

    first_length.append(len(i))
    second_length.append(len(j))
    
    text = ['[CLS]']+i+['[SEP]']+j+['[SEP]']
    full_text.append(tokenizer.convert_tokens_to_ids(text))
    attention_mask.append(np.ones_like(text))

    token_type_id = [0]*(len(i)+2) + [1]*(len(j)+1)
    token_type_ids.append(token_type_id)

  all_pad_tokens = pad_sequences(full_text, maxlen=max_len*2, padding='post', truncating='post')
  all_mask = pad_sequences(attention_mask, maxlen=max_len*2, padding='post', truncating='post')
  token_type_ids = pad_sequences(token_type_ids, maxlen=max_len*2, padding='post', truncating='post')
  
  return {
      'input_ids': torch.tensor(all_pad_tokens,dtype=torch.long),
      'attention_mask': torch.tensor(all_mask, dtype=torch.long),
      'token_type_ids':torch.tensor(token_type_ids,dtype = torch.long),
      'mpp_label':torch.tensor(mpp_label.values,dtype = torch.float),
      'ml_label':torch.tensor(ml_label.values,dtype = torch.float),
      'first_all_value': torch.tensor(first_all_value.values, dtype=torch.float),
      'second_all_value': torch.tensor(second_all_value.values, dtype=torch.float),
      'first_sen_len':torch.tensor(first_length,dtype = torch.long),
      'second_sen_len':torch.tensor(second_length,dtype = torch.long)
      
  }

    

In [7]:
class BertNerClassifier(nn.Module):

  def __init__(self, n_classes_cls,n_classes_reg):
    super(BertNerClassifier, self).__init__()
    self.bert = AutoModel.from_pretrained(PRETRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.1)
    self.out_cls = nn.Linear(self.bert.config.hidden_size, n_classes_cls)
    

    self.out_reg = nn.Linear(self.bert.config.hidden_size, n_classes_reg)
    

  
  def forward(self, input_ids, attention_mask,token_type_ids,max_len,length1,length2):
    output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask,
      token_type_ids = token_type_ids

    )
    cls_output = []
    sent1_output = []
    sent2_output = []

    for last_output,len1,len2 in zip(output[0],length1,length2):
      
      
      sent1_output.append(torch.mean(last_output[1:(len1+1)],dim=0))
      sent2_output.append(torch.mean(last_output[len1+2:(len1+2+len2)],dim=0))

    cls_output = output[1]
    sent1_output = torch.stack(sent1_output)
    
    sent2_output = torch.stack(sent2_output)
    
    
    cls_output = self.out_cls(cls_output)
    
    
    
    return cls_output,self.out_reg(sent1_output),self.out_reg(sent2_output)

In [8]:
 
class DataSequence(Dataset):

    def __init__(self, data):

      self.features = data['input_ids']
      self.mask = data['attention_mask']
      self.token_type_ids = data['token_type_ids']
      self.mpp_label = data['mpp_label']
      self.ml_label = data['ml_label']
      self.first_all_value = data['first_all_value']
      self.second_all_value = data['second_all_value']
      self.first_sen_len = data['first_sen_len']   
      self.second_sen_len = data['second_sen_len']   

    def __len__(self):

        return len(self.mpp_label)

    def __getitem__(self, index):

        return self.features[index],self.mask[index],self.token_type_ids[index],self.mpp_label[index],self.ml_label[index],self.first_all_value[index],self.second_all_value[index],self.first_sen_len[index],self.second_sen_len[index]

In [9]:
def generate_dataloader(post1,post2,mpp_label,ml_label,max_len,mpp_value1,ml_value1,mpp_value2,ml_value2,batch_size):
  data = generate_input_data(post1,post2,mpp_label,ml_label,max_len,mpp_value1,ml_value1,mpp_value2,ml_value2)
  data = DataSequence(data)
  dataloader = DataLoader(data, num_workers=10, batch_size=batch_size)
  
  return dataloader


In [10]:
EPOCHS = 30


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
loss_reg = nn.MSELoss().to(device)
loss_cls = nn.BCEWithLogitsLoss().to(device)






In [11]:

def train_and_eval(model, 
  train_loader, 
  validation_loader, 
  optimizer,  
  device,
  epoch,
  model_name
  ):
  model.train()
  max_len = 256
  global acccuracy_state
  for epo in range(epoch):
    
    train_loss = []
    
    global train_mpp_predictions 
    train_mpp_predictions = []
    global train_ml_predictions
    train_ml_predictions = []
    

    validation_loss = []
    
    validation_mpp_predictions = []
    validation_ml_predictions = []


    global real_train_mpp 
    real_train_mpp = []
    global real_train_ml
    real_train_ml = []
    real_validation_mpp = []
    real_validation_ml = []
    
   
    
    for data in train_loader:
      
      input_ids = data[0].to(device)
      attention_mask = data[1].to(device)
      token_type_ids = data[2].to(device)
      
      mpp_targets = data[3].to(device)
      ml_targets = data[4].to(device)
      first_targets = data[5].to(device)
      second_targets = data[6].to(device)
      first_sen_len = data[7].to(device)
      second_sen_len = data[8].to(device)
      
      

      output_cls,output_sen1,ouptut_sen2 = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        token_type_ids = token_type_ids,
        max_len = max_len,
        length1 = first_sen_len,
        length2 = second_sen_len
   
      )
      mpp_ml_label = torch.stack((mpp_targets,ml_targets),1)
      
      
      loss1 = loss_cls(output_cls.view(-1),mpp_ml_label.view(-1)).to(device)
      
      loss2 = loss_reg(output_sen1.view(-1),first_targets.view(-1)).to(device)
      
      loss3 = loss_reg(ouptut_sen2.view(-1),second_targets.view(-1)).to(device)
      
      loss = loss1+10*loss2+10*loss3
 

      train_loss.append(loss.item())
      
      
      
      for i in output_cls:
        if i[0]>0:
          train_mpp_predictions.append(1)
        else:
          train_mpp_predictions.append(0)
        if i[1]>0:
          train_ml_predictions.append(1)
        else:
          train_ml_predictions.append(0)

      for i,j in zip(mpp_targets,ml_targets):
        real_train_mpp.append(i.cpu().item())
        real_train_ml.append(j.cpu().item())
    
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    model.eval()

    with torch.no_grad():
      for data in validation_loader:
    
        input_ids = data[0].to(device)
        attention_mask = data[1].to(device)
        token_type_ids = data[2].to(device)
      
        mpp_targets = data[3].to(device)
        ml_targets = data[4].to(device)
        first_targets = data[5].to(device)
        second_targets = data[6].to(device)
        first_sen_len = data[7].to(device)
        second_sen_len = data[8].to(device)
      

        output_cls,output_sen1,ouptut_sen2 = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        token_type_ids = token_type_ids,
        max_len = max_len,
        length1 = first_sen_len,
        length2 = second_sen_len
   
        )
        mpp_ml_label = torch.stack((mpp_targets,ml_targets),1)     
      
        loss1 = loss_cls(output_cls.view(-1),mpp_ml_label.view(-1)).to(device)
        
        loss2 = loss_reg(output_sen1.view(-1),first_targets.view(-1)).to(device)
        
        loss3 = loss_reg(ouptut_sen2.view(-1),second_targets.view(-1)).to(device)
        
        loss = loss1+10*loss2+10*loss3

        validation_loss.append(loss.item())
      
        for i in output_cls:
          if i[0]>0:
            validation_mpp_predictions.append(1)
          else:
            validation_mpp_predictions.append(0)
          if i[1]>0:
            validation_ml_predictions.append(1)
          else:
            validation_ml_predictions.append(0)

        for i,j in zip(mpp_targets,ml_targets):
          real_validation_mpp.append(i.cpu().item())
          real_validation_ml.append(j.cpu().item())
    
    train_mpp_accuracy = accuracy_score(real_train_mpp, train_mpp_predictions)
    train_ml_accuracy = accuracy_score(real_train_ml, train_ml_predictions)
    validation_mpp_accuracy = accuracy_score(real_validation_mpp, validation_mpp_predictions)
    validation_ml_accuracy = accuracy_score(real_validation_ml, validation_ml_predictions)
    
    
    final_train_loss = np.mean(train_loss)
 
    final_validation_loss = np.mean(validation_loss)

    average_accuracy = (validation_mpp_accuracy+validation_ml_accuracy)/2

    if average_accuracy > acccuracy_state:
      acccuracy_state =  average_accuracy
      torch.save(model.state_dict(), model_name+'double_value_cls.bin')



    print(
'Epoch [{}] Train Totol Loss: {:.4f}, Train MPP Accuracy: {:.4f}, Train ML Accuracy: {:.4f}, Valid Total Loss: {:.4f}, Valid MPP Accuracy: {:.4f}, Valid ML Accuracy: {:.4f}'
    .format(epo+1,final_train_loss, train_mpp_accuracy, train_ml_accuracy,final_validation_loss,validation_mpp_accuracy,validation_ml_accuracy))

In [12]:
def balance_dataset(dataset):
  new_dataset = deepcopy(dataset)
 
    
  new_dataset[['post1', 'post2']] = new_dataset[['post2', 'post1']]
  new_dataset[['post1_en', 'post2_en']] = new_dataset[['post2_en', 'post1_en']]
  new_dataset[['MPP1', 'MPP2']] = new_dataset[['MPP2', 'MPP1']]
  new_dataset[['ML1', 'MPP2']] = new_dataset[['ML2', 'MPP1']]
  new_dataset['ML_label'] = new_dataset['ML_label'].apply(lambda x: swich_value(x))
  new_dataset['MPP_label'] = new_dataset['MPP_label'].apply(lambda x: swich_value(x))
  dataset = pd.concat([new_dataset,dataset])


  return dataset

In [13]:
def swich_value(x):
  if x == 0:
    x = 1
  else:
    x = 0
  return x

In [14]:
%cd /content/drive/MyDrive/model_result/ERAI Dataset_with_translation

/content/drive/MyDrive/model_result/ERAI Dataset_with_translation


In [ ]:
random_seed = [171,354,550,667,985]
model_name = ['roberta','chi','fin','sec']

for i in model_name:

  if i == 'chi':
    PRETRAINED_MODEL_NAME = "bert-base-chinese"
    tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
  if i == 'roberta':
    PRETRAINED_MODEL_NAME = '/content/drive/MyDrive/ROBERT_4_model.bin'
    tokenizer = BertTokenizer.from_pretrained("hfl/chinese-roberta-wwm-ext")
  if i == 'fin':
    PRETRAINED_MODEL_NAME = 'yiyanghkust/finbert-tone'
    tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
  if i == 'sec':
    PRETRAINED_MODEL_NAME = "nlpaueb/sec-bert-shape"
    tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

  acccuracy_state = 0


  for j in random_seed:
    print('model_name '+i)
    print('seed '+str(j))
    df = pd.read_json('ERAI_Dataset_pairwise_train_with_translation.json')
    df.reset_index(drop = True,inplace = True)
    df = df.dropna(axis = 0)
    df = shuffle(df,random_state=j)
    df['post1_en']=df['post1_en'].apply(lambda x:x.replace('\n',''))
    df['post2_en']=df['post2_en'].apply(lambda x:x.replace('\n',''))

    df['post1']=df['post1'].apply(lambda x:x.replace('\n',''))
    df['post2_']=df['post2'].apply(lambda x:x.replace('\n',''))

    
    train = df[0:156]
    
    train = balance_dataset(train)
    print(train['MPP_label'].value_counts())
    

    test = df[156:200]
    test = balance_dataset(test)
    print(test['MPP_label'].value_counts())
    

    if i=='sec'or i=='fin':

      train_post1 = train['post1_en']
      train_post2 = train['post2_en']
      test_post1 = test['post1_en']
      test_post2 = test['post2_en']
    else:
      train_post1 = train['post1']
      train_post2 = train['post2']
      test_post1 = test['post1']
      test_post2 = test['post2']
  
    train_mpp_label = train['MPP_label']
    train_ml_label = train['ML_label']
    train_mpp_value1 = train['MPP1'] 
    train_mpp_value2 = train['MPP2']
    train_ml_value1 = train['ML1']
    train_ml_value2 = train['ML2']

    
    test_mpp_label = test['MPP_label']
    test_ml_label = test['ML_label']
    test_mpp_value1 = test['MPP1'] 
    test_mpp_value2 = test['MPP2']
    test_ml_value1 = test['ML1']
    test_ml_value2 = test['ML2']

    max_len = 256
    batch_size = 16
    train_dataloader = generate_dataloader(train_post1,train_post2,train_mpp_label,train_ml_label,max_len,train_mpp_value1,train_ml_value1,train_mpp_value2,train_ml_value2,batch_size)
    test_dataloader = generate_dataloader(test_post1,test_post2,test_mpp_label,test_ml_label,max_len,test_mpp_value1,test_ml_value1,test_mpp_value2,test_ml_value2,batch_size)

    label_num_cls = 2
    label_num_reg = 2
    model= BertNerClassifier(label_num_cls,label_num_reg).to(device)
    optimizer = AdamW(model.parameters(), lr=2e-6, correct_bias=False)
    
    
    train_and_eval(model, 
      train_dataloader, 
      test_dataloader, 
      optimizer,  
      device,
      EPOCHS,
      i
      )




In [ ]:
model.load_state_dict(torch.load('double_value_cls.bin', map_location=torch.device("cuda:0")))

In [ ]:
def generate_test_data(post1,post2,max_len):
 
  full_text = []
  attention_mask = []
  token_type_ids = []
  first_length = []
  second_length = []
  for i,j in zip(post1,post2):
    if len(i)>max_len:
      i = i[:max_len]
    if len(j)>max_len:
      j = j[:max_len]
    
    i = tokenizer.tokenize(i)
    j = tokenizer.tokenize(j)

    first_length.append(len(i))
    second_length.append(len(j))
    
    text = ['[CLS]']+i+['[SEP]']+j+['[SEP]']
    full_text.append(tokenizer.convert_tokens_to_ids(text))
    attention_mask.append(np.ones_like(text))

    token_type_id = [0]*(len(i)+2) + [1]*(len(j)+1)
    token_type_ids.append(token_type_id)

  all_pad_tokens = pad_sequences(full_text, maxlen=max_len*2, padding='post', truncating='post')
  all_mask = pad_sequences(attention_mask, maxlen=max_len*2, padding='post', truncating='post')
  token_type_ids = pad_sequences(token_type_ids, maxlen=max_len*2, padding='post', truncating='post')
  
  return {
      'input_ids': torch.tensor(all_pad_tokens,dtype=torch.long),
      'attention_mask': torch.tensor(all_mask, dtype=torch.long),
      'token_type_ids':torch.tensor(token_type_ids,dtype = torch.long),
      'first_sen_len':torch.tensor(first_length,dtype = torch.long),
      'second_sen_len':torch.tensor(second_length,dtype = torch.long)
      
  }

In [ ]:
 
class testSequence(Dataset):

    def __init__(self, data):

      self.features = data['input_ids']
      self.mask = data['attention_mask']
      self.token_type_ids = data['token_type_ids']
      self.first_sen_len = data['first_sen_len']   
      self.second_sen_len = data['second_sen_len']   

    def __len__(self):

        return len(self.features)

    def __getitem__(self, index):

        return self.features[index],self.mask[index],self.token_type_ids[index],self.first_sen_len[index],self.second_sen_len[index]

In [ ]:
def generate_test_dataloader(post1,post2,max_len,batch_size):
  data = generate_test_data(post1,post2,max_len)
  data = testSequence(data)
  dataloader = DataLoader(data, num_workers=10, batch_size=batch_size)
  
  return dataloader

In [ ]:
test_post1 = df2['post1']
test_post2 = df2['post2']
test_loader = generate_test_dataloader(test_post1,test_post2,256,16)

In [ ]:
def real_prediction(model, 
  test_dataloader,  
  optimizer,
  device
  ):
  
  test_mpp_predictions = []
  test_ml_predictions = []
  
  model.eval()
  with torch.no_grad():
    for data in test_dataloader:
    
        input_ids = data[0].to(device)
        attention_mask = data[1].to(device)
        token_type_ids = data[2].to(device)
      
        
        first_sen_len = data[3].to(device)
        second_sen_len = data[4].to(device)
      

        output_cls,output_sen1,ouptut_sen2 = model(
        input_ids=input_ids,
        attention_mask=attention_mask,
        token_type_ids = token_type_ids,
        max_len = max_len,
        length1 = first_sen_len,
        length2 = second_sen_len
   
        )
     
        
      
        for i in output_cls:
          if i[0]>0:
            test_mpp_predictions.append(1)
          else:
            test_mpp_predictions.append(0)
          if i[1]>0:
            test_ml_predictions.append(1)
          else:
            test_ml_predictions.append(0)
        
  return test_mpp_predictions,test_ml_predictions
      



In [ ]:
test_mpp_predictions,test_ml_predictions = real_prediction(model,test_loader,optimizer,device)

In [ ]:
df2['MPP_prediction'] = test_mpp_predictions
df2['ML_prediction'] = test_ml_predictions
df2.to_json('ERAI_pairwise_[aiml]_[1].json')

In [ ]:
df2